In [1]:
print("OK!")

OK!


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


d:\anaconda\envs\gkvchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [38]:
%pwd

'D:\\GKV chatbot\\GKV-Chatbot-using-Llama2-main'

In [39]:
import os
os.chdir("../")

In [40]:
%pwd

'D:\\GKV chatbot'

In [4]:
import os  

# Change to the specified directory  
os.chdir("D:\GKV chatbot\GKV-Chatbot-using-Llama2-main")

In [5]:
extracted_data = load_pdf("data/")

In [43]:
#extracted_data

In [6]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 1646


In [46]:
#text_chunks

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [51]:
#query_result

In [12]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')


In [15]:
#Initializing the Pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "gkvchatbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '318b9900fed050814476442a22ef1baf', 'Date': 'Mon, 25 Nov 2024 03:34:32 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [13]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,)

In [17]:

# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings)
query = " What is the hostel fee?"
docs = docsearch.similarity_search(query,k=20)
print("Result",docs)

Result [Document(metadata={'page': 3.0, 'source': 'data\\qa.pdf'}, page_content='4 \n \nProvisional Certificate fee                200/-     \nDuplicate Migration fee                 400/-     \nRepeat Examination fee per paper           500/-     \nCharacter Certificate fee                 50/-     \nRe-admission fee                     200/-    \n \nQ 12. Whether hostel facility is available or not?    \nA. Hostel facility is available in the main campus of Vishwavidyalaya for B.Tech. Ist  Year \nstudents only against limited seats.  \n \nQ 13. What is the hostel fee?'), Document(metadata={'page': 3.0, 'source': 'data\\qa.pdf'}, page_content='4 \n \nProvisional Certificate fee                200/-     \nDuplicate Migration fee                 400/-     \nRepeat Examination fee per paper           500/-     \nCharacter Certificate fee                 50/-     \nRe-admission fee                     200/-    \n \nQ 12. Whether hostel facility is available or not?    \nA. Hostel facility

In [ ]:
# Refined prompt for detailed responses
prompt_template = """
Use the following context to answer the user's question thoroughly.
Ensure your answer is detailed and includes examples or additional explanations where relevant.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}


Helpful Answer:
"""

In [ ]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoModelForSeq2SeqLM  
import os  

# Define your model and desired path  
model_name = "google/flan-t5-large"  # Replace with your desired model   
desired_path = "D:/GKV chatbot/GKV-Chatbot-using-Llama2-main/model"  # Replace with your specific path  

# Ensure the directory exists  
os.makedirs(desired_path, exist_ok=True)  

# Download the model and tokenizer to the specified path  
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, cache_dir=desired_path)  
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=desired_path)  

print("Model downloaded to:", desired_path)

Model downloaded to: D:/GKV chatbot/GKV-Chatbot-using-Llama2-main/model


In [ ]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature": 0.5, "max_length": 512})

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 20}),  # Increase retrieval size
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True,
)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])